In [71]:
# Import all necessary libraries
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import KFold
import sklearn.metrics as metrics



## Load Data 

In [72]:
data = pd.read_csv("train_features_normalized.csv")
DATA = np.array(data.iloc[:, 2:].values, dtype=float)
DATA = DATA.reshape(227940//12,35*12)

data_bin = pd.read_csv("train_labels.csv")
DATA_BIN = np.array(data_bin.iloc[:, 1:].values, dtype=float)

test = pd.read_csv("test_features_normalized.csv")
TEST = np.array(test.iloc[:, 2:].values, dtype=float)
TEST = TEST.reshape(151968//12,35*12)

print(DATA.shape)
print(DATA_BIN.shape)

col_names = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2', 'LABEL_Bilirubin_direct', 'LABEL_EtCO2', 'LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']







(18995, 420)
(18995, 15)


## Part 1 and part 2

In [73]:
data = []

for i in range(0, 10):
    lr = LogisticRegression(max_iter=1000)
    lr.fit(DATA, DATA_BIN[:, i]) #train our logistic regression

    Y_pred_lr = lr.predict_proba(TEST)[:,1]
    data.append(Y_pred_lr) # we run on our test and put it in the list of arrays






## Part 3: Linear Regression

In [74]:
output_train = DATA_BIN[:, 11:]

inputs_dim = DATA.shape[1]
outputs_dim = output_train.shape[1]

print(inputs_dim)
print(outputs_dim)

model = tf.keras.Sequential([
  tf.keras.layers.Dense(units=100, input_shape=[inputs_dim], activation='relu'),
  tf.keras.layers.Dense(units=25, activation='relu'),
  tf.keras.layers.Dense(units=outputs_dim)
])

model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.1), metrics=['mean_squared_error'])
model.fit(DATA, output_train, epochs=20)

pred = model.predict(TEST)



420
4
Epoch 1/20
594/594 [==============================] - 1s 2ms/step - loss: 96.8364 - mean_squared_error: 96.8364
Epoch 2/20
594/594 [==============================] - 1s 2ms/step - loss: 52.7871 - mean_squared_error: 52.7871
Epoch 3/20
594/594 [==============================] - 1s 2ms/step - loss: 49.2722 - mean_squared_error: 49.2722
Epoch 4/20
594/594 [==============================] - 1s 2ms/step - loss: 48.6559 - mean_squared_error: 48.6559
Epoch 5/20
594/594 [==============================] - 1s 2ms/step - loss: 46.8238 - mean_squared_error: 46.8238
Epoch 6/20
594/594 [==============================] - 1s 2ms/step - loss: 49.5169 - mean_squared_error: 49.5169
Epoch 7/20
594/594 [==============================] - 1s 2ms/step - loss: 45.6270 - mean_squared_error: 45.6270
Epoch 8/20
594/594 [==============================] - 1s 2ms/step - loss: 46.5884 - mean_squared_error: 46.5884
Epoch 9/20
594/594 [==============================] - 1s 2ms/step - loss: 46.4208 - mean_squared_e

In [75]:
data.extend(pred.transpose().tolist())
print(len(data))

14


## Save as CSV and Zipped

In [77]:
np_array = np.array(data).transpose()
new_df = pd.DataFrame(np_array, columns=col_names)


output_csv = pd.read_csv("output.csv")
output_csv.update(new_df)
compression_opts = dict(method='zip',
                        archive_name='out.csv', )  
output_csv.to_csv('out.zip', index=False,
          compression=compression_opts, float_format='%.3f')